In [ ]:
import cv2
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.applications import MobileNetV2
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization,GlobalAveragePooling2D
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ReduceLROnPlateau


# Load Data

In [ ]:
TRAIN_SIZE = 101 #60000
train=pd.read_csv('../input/Kannada-MNIST/train.csv')
train.head

In [ ]:
train.shape

In [ ]:
label_value_cnts=train.label.value_counts()
label_value_cnts

In [ ]:
def x_train_28_28_1():
    X_train=train.drop('label',axis=1)
    X_train=X_train/255 #normalize
    return X_train.values[:TRAIN_SIZE, :].reshape(-1,28,28,1)

# PreProcessing Image: from (28, 28, 1) to (94, 94, 3)

In [ ]:
def convert_shape_channel(X):
    X = np.float32(X)
    n = X.shape[0]
    X_rgb = np.zeros((n, 94, 94, 3))
    for i in range(n):        
        resized  = cv2.resize(X[i], dsize=(94, 94), interpolation=cv2.INTER_CUBIC)
        rgb = cv2.cvtColor(resized,cv2.COLOR_GRAY2RGB) 
        X_rgb[i] = rgb   
    
    return X_rgb.reshape((n, 94, 94, 3))


In [ ]:
X_train = x_train_28_28_1()
print(X_train.shape)
X_train = convert_shape_channel(X_train)
print(X_train.shape)

In [ ]:
X_train.shape

In [ ]:
Y_train=train.label[:TRAIN_SIZE]
Y_train=to_categorical(Y_train)

In [ ]:
Y_train.shape

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_train,Y_train,random_state=42,test_size=0.2)

# Data Augementation

In [ ]:
datagen = ImageDataGenerator( 
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1)  


datagen.fit(X_train)

In [ ]:
X_train

# Model

In [ ]:
def mobilenet():
    base_model = MobileNetV2(
        weights = 'imagenet',
        input_shape = (96, 96, 3),
        include_top = False
    )
    base_model.trainable = False
    return Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.4),
        Dense(10, activation = 'softmax')
    ])

In [ ]:
def simple_model():
    model = Sequential()
    

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization(momentum=.15))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))


    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization(momentum=0.15))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(BatchNormalization(momentum=.15))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))


    model.add(Flatten())
    model.add(Dense(128, activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation = "softmax"))
    return simple_model()

In [ ]:
model = mobilenet()

In [ ]:
model.summary()

In [ ]:
optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999)

In [ ]:
model.compile(optimizer=optimizer,loss=['categorical_crossentropy'],metrics=['accuracy'])

# Learning rate reduction

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
epochs=5
batch_size=64 if TRAIN_SIZE > 100 else 1

# Train

In [ ]:
history = model.fit_generator(
    datagen.flow(X_train,y_train, batch_size=batch_size), 
    epochs = epochs, 
    validation_data = (X_test,y_test),
    verbose = 2, 
    steps_per_epoch=X_train.shape[0] // batch_size, 
    callbacks=[learning_rate_reduction]
)

In [ ]:
y_pre_test=model.predict(X_test)
y_pre_test=np.argmax(y_pre_test,axis=1)
y_test=np.argmax(y_test,axis=1)

In [ ]:
conf=confusion_matrix(y_test,y_pre_test)
conf=pd.DataFrame(conf,index=range(0,10),columns=range(0,10))
conf

# Test

In [ ]:
test=pd.read_csv('../input/Kannada-MNIST/test.csv')
test=test.drop('id',axis=1)
test=test/255
test=test.values.reshape(-1,28,28,1)

In [ ]:
test = convert_shape_channel(test)

In [ ]:
y_pre=model.predict(test)     
y_pre=np.argmax(y_pre,axis=1) 

In [ ]:
submission=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission['label']=y_pre
submission.to_csv('submission.csv',index=False)